# Custom Validator

In [1]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, obj, owner):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

In [2]:
class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {self.public_name} to be one of {self.options}!')

In [3]:
class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {self.public_name} {value} to be an int or float not {value.__class__.__name__}')
        if (self.minvalue is not None) and (value <= self.minvalue):
            raise ValueError(
                f'Expected {self.public_name} {value} to be greater than {self.minvalue}'
            )
        if (self.maxvalue is not None) and (value >= self.maxvalue):
            raise ValueError(
                f'Expected {self.public_name} {value} to be less than {self.maxvalue}'
            )

In [4]:
class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {self.public_name} {value} to be an str not {value.__class__.__name__}')
        if (self.minsize is not None) and (len(value) < self.minsize):
            raise ValueError(
                f'Expected "{self.public_name}" {value} to be no smaller than {self.minsize}'
            )
        if (self.maxsize is not None) and (len(value) > self.maxsize):
            raise ValueError(
                f'Expected "{self.public_name}" {value} to be no greater than {self.maxsize}'
            )
        if (self.predicate is not None) and (not self.predicate(value)):
            raise ValueError(
                f'Expected {self.predicate} to be true for {value!r}'
            )

In [5]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0, maxvalue=10)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

In [6]:
# this should raise `ValueError`
Component(name='Widgetbdfbdbdbbdbbsb', kind='metal', quantity=5)

ValueError: Expected "name" Widgetbdfbdbdbbdbbsb to be no greater than 10

In [7]:
# this should raise `ValueError`
Component(name='WIDGET', kind='metle', quantity=5)

ValueError: Expected kind to be one of {'plastic', 'wood', 'metal'}!

In [8]:
# this should raise `ValueError`
Component(name='WIDGET', kind='metal', quantity=-10)

ValueError: Expected quantity -10 to be greater than 0

In [9]:
# this should run successfully with no error
Component('WIDGET', 'metal', 5)